In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.layers import LSTM, GRU
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam, SGD, RMSprop
from sklearn.preprocessing import MinMaxScaler

In [2]:
dataset = pd.read_csv('C:/Users/diana/Desktop/proiect test/energy_client/client_1.csv')
dataset = dataset.drop(columns=['Data Start Time', 'Meter No.'])
dataset['Data End Time'] = pd.to_datetime(dataset['Data End Time'], format='%Y-%m-%d %H:%M:%S')
dataset = dataset.set_index('Data End Time')
dataset = dataset.sort_index()
dataset = dataset.asfreq('15min')
dataset = dataset.rename(columns={'Active energy import A+(1.8.0)': 'Demand'})
dataset.head()

,Demand
Data End Time,
2023-05-15 00:15:00,0.128
2023-05-15 00:30:00,0.128
2023-05-15 00:45:00,0.128
2023-05-15 01:00:00,0.112
2023-05-15 01:15:00,0.097


In [3]:
dataset = dataset.interpolate()

In [4]:
dataset.dtypes

Demand    float64
dtype: object

In [5]:
train = dataset.iloc[:537]
test = dataset.iloc[537:]

In [6]:
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [7]:
from keras.preprocessing.sequence import TimeseriesGenerator
 
n_input = 500
n_features = 1
generator = TimeseriesGenerator(scaled_train,
                                scaled_train,
                                length=n_input,
                                batch_size=1)
X, y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')
# We do the same thing, but now instead for 12 months
n_input = 12
generator = TimeseriesGenerator(scaled_train,
                                scaled_train,
                                length=n_input,
                                batch_size=1)

Given the Array: 
[1.20075047e-01 1.20075047e-01 1.20075047e-01 1.05065666e-01
 9.09943715e-02 9.75609756e-02 8.72420263e-02 8.25515947e-02
 6.19136961e-02 9.09943715e-02 1.05065666e-01 9.00562852e-02
 9.28705441e-02 1.00375235e-01 1.10694184e-01 7.69230769e-02
 6.84803002e-02 7.87992495e-02 8.63039400e-02 8.72420263e-02
 6.09756098e-02 9.56848030e-02 1.01313321e-01 1.03189493e-01
 8.91181989e-02 7.41088180e-02 8.63039400e-02 4.78424015e-02
 2.53283302e-02 3.56472795e-02 6.75422139e-02 4.97185741e-02
 7.69230769e-02 7.03564728e-02 8.25515947e-02 4.22138837e-02
 6.84803002e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.97185741e-02
 3.06754221e-01 4.00562852e-01 6.24765478e-01 6.06941839e-01
 5.48780488e-01 4.98123827e-01 3.55534709e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.53283302e-02 3.09568480e-02 3.34896811e-01 1.71669794e-01
 3.09568480e-02 3.65853659e-02 0.00000000e+00 0.00000000e+00
 2.063

In [8]:
model = Sequential()
model.add(LSTM(131, activation='relu',
               input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()
model.fit(generator, epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 131)               69692     
                                                                 
 dense (Dense)               (None, 1)                 132       
                                                                 
Total params: 69,824
Trainable params: 69,824
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
525/525 [==============================] - 9s 12ms/step - loss: 0.0102
Epoch 2/5
525/525 [==============================] - 6s 12ms/step - loss: 0.0079
Epoch 3/5
525/525 [==============================] - 6s 11ms/step - loss: 0.0065
Epoch 4/5
525/525 [==============================] - 5s 9ms/step - loss: 0.0059
Epoch 5/5
525/525 [==============================] - 3s 6ms/step - loss: 0.0059
